In [23]:
import polars as pl
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler

df = pl.read_csv("../Data/Titanic/train.csv")
df_kaggle = pl.read_csv("../Data/Titanic/test.csv")

In [24]:
def standardize(column):
    return (column - column.mean()) / column.std()

In [ ]:
def preprocessing(df):
    df = df.with_columns(
        [
            pl.when(pl.col("Sex")=="male").then(pl.lit(1)).otherwise(pl.lit(0)).alias("Sex"),
            pl.col("Age").fill_null(pl.col("Age").median()),
            pl.col("Fare").log1p().alias("Fare"),
        ]
    ).to_dummies(
        "Embarked"
    ).drop(
        [
            "PassengerId",
            "Name",
            "Ticket",
            "Cabin",
            "Embarked_S",
            "Embarked_null",
        ],
        strict=False
    )

    df = df.with_columns(
        [
            standardize(pl.col("Age")).alias("Age"),
            standardize(pl.col("Fare")).alias("Fare")
        ]
    )
    return df

In [26]:
df = preprocessing(df)
df_kaggle = preprocessing(df_kaggle)

df.head()
df_kaggle.head()

NameError: name 'false' is not defined

In [ ]:
df.write_csv("../Data/Titanic/baseDataframe.csv")
df_kaggle.write_csv("../Data/Titanic/baseDataframeSolution.csv")